In [ ]:
# point to root dir
import os

os.chdir('../')
os.getcwd()

In [ ]:
# imports
import numpy as np
import pandas as pd
from celer import GroupLasso, celer_path

In [ ]:
# response
y = pd.Series([0.2, 2.5, 1.1, 3.7])

# features
X = pd.DataFrame({
    'c': [1.2, 2.1, 3.5, 4.7],
    'is_good': ['yes', 'no', 'yes', 'no'],
    'color': ['red', 'red', 'white', 'blue']
})

In [ ]:
from py_scripts.processing import adapt_to_group_lasso

X_with_dummies, groups = adapt_to_group_lasso(X)

In [ ]:
n_groups = len(groups)
grp_indices = np.append([0], np.cumsum(groups))

n_alphas = 100
alphas, coefs, _ = celer_path(X_with_dummies.values, y.values.flatten(),
                              pb='grouplasso', groups=groups,
                              n_alphas=n_alphas, eps=1e-6, prune=1)

In [ ]:
variables_colors: dict[int, str] = {
    0: '#636EFA',
    1: '#EF553B', 2: '#EF553B',
    3: '#00CC96', 4: '#00CC96', 5: '#00CC96'
}
variables_grp: dict[int, str] = {
    0: 'c',
    1: 'is_good', 2: 'is_good',
    3: 'color', 4: 'color', 5: 'color' 
}
variables_showlegend: dict[int, bool] = {
    0: True,
    1: True, 2: False,
    3: True, 4: False, 5: False
}

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


pio.templates.default = "plotly_white"

fig = go.Figure(
    data=[
        go.Scatter(
            x=alphas / alphas[0],
            y=coefs[i, :],
            mode="lines",
            marker_color=variables_colors[i],
            name=variables_grp[i],
            showlegend=variables_showlegend[i]
        )
    for i in range(6)]
)


fig.update_layout(
    xaxis={
        'title': 'alpha / alpha_max',
        'type': 'log'
    },
    yaxis_title="coefficients values",
    width=800
)
fig.show()

In [ ]:
from py_scripts.analytics import score_features

vars_scores = score_features(X, y)

In [ ]:
vars_scores

In [ ]:

fig = go.Figure(
    data=[
        go.Bar(
            x=[ele[0] for ele in vars_scores],
            y=[ele[1] for ele in vars_scores],
            marker_color=['#636EFA', '#EF553B', '#00CC96']
        )
    ]
)

fig.update_layout(
    xaxis_title='variables',
    yaxis={
        'title': 'scores', 
        'type': 'log'
    },
    width=800
)

fig.show()